In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import pickle
import collections
import random
import csv
import json
import re
from datetime import datetime
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import seaborn as sns
from nltk.corpus import nps_chat
from nltk.corpus import webtext
from sklearn.model_selection import train_test_split
import spacy
import pickle
import nltk
import itertools
import sklearn
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.externals import joblib
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')

%matplotlib inline

ModuleNotFoundError: No module named 'lightgbm'

$a_3=4$

# Using Light GBM to Classify Kaggle MBTI DataSet 

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer,CountVectorizer
from scipy.sparse import hstack,csr_matrix
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [3]:
from sklearn.preprocessing import LabelEncoder

unique_type_list = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
lab_encoder = LabelEncoder().fit(unique_type_list)

In [4]:
data = pd.read_csv('../../data/mbti_1.csv')
print(len(data))
data.head()

8675


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [5]:
import time
##### Compute list of subject with Type | list of comments 
# Time
%time data.posts[1].replace('+', ' ').replace('.', ' ').replace(',', ' ').replace(':', ' ')
%time re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', data.posts[1])

from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Lemmatizer | Stemmatizer
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()

# Cache the stop words for speed 
cachedStopWords = stopwords.words("english")

# One post
OnePost = data.posts[1]
# List all urls
urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', OnePost)
# Remove urls
temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'link', OnePost)
# Keep only words
temp = re.sub("[^a-zA-Z]", " ", temp)
# Remove spaces > 1
temp = re.sub(' +', ' ', temp).lower()
# Remove stopwords and lematize
%time stemmer.stem(" ".join([w for w in temp.split(' ') if w not in cachedStopWords]))

print("\nBefore preprocessing:\n\n", OnePost[0:500])
print("\nAfter preprocessing:\n\n", temp[0:500])
print("\nList of urls:")
print(len(urls))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 720 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 380 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.54 ms

Before preprocessing:

 'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin*  That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This + L

After preprocessing:

  i m finding the lack of me in these posts very alarming sex can be boring if it s in the same position often for example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary there isn t enough giving new 

In [6]:
##### Compute list of subject with Type | list of comments 
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Lemmatize
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()

def pre_process_data(data, remove_stop_words=True):

    list_personality = []
    list_posts = []
    list_type=[]
    len_data = len(data)
    i=0
    
    for row in data.iterrows():
        i+=1
        if i % 500 == 0:
            print("%s | %s rows" % (i, len_data))

        ##### Remove and clean comments
        posts = row[1].posts
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'link', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in cachedStopWords])
        else:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])

        type_labelized = lab_encoder.transform([row[1].type])[0]
        list_personality.append(type_labelized)
        list_posts.append(temp)
        list_type.append(row[1].type)

    #del data
    list_posts = np.array(list_posts)
    list_personality = np.array(list_personality)
    return list_posts, list_personality,list_type

list_posts, list_personality,list_type = pre_process_data(data, remove_stop_words=True)

500 | 8675 rows
1000 | 8675 rows
1500 | 8675 rows
2000 | 8675 rows
2500 | 8675 rows
3000 | 8675 rows
3500 | 8675 rows
4000 | 8675 rows
4500 | 8675 rows
5000 | 8675 rows
5500 | 8675 rows
6000 | 8675 rows
6500 | 8675 rows
7000 | 8675 rows
7500 | 8675 rows
8000 | 8675 rows
8500 | 8675 rows


In [7]:
prep_data=pd.DataFrame({'content':list_posts,'label':list_personality,'type':list_type})
prep_data['length']=prep_data['content'].apply(lambda x:len(x))
prep_data['IE']=prep_data['type'].apply(lambda x:int(x[0]!='I'))
prep_data['NS']=prep_data['type'].apply(lambda x:int(x[1]!='N'))
prep_data['TF']=prep_data['type'].apply(lambda x:int(x[2]!='T'))
prep_data['JP']=prep_data['type'].apply(lambda x:int(x[3]!='J'))
print(prep_data.shape)
prep_data.head()

(8675, 8)


,content,label,type,length,IE,NS,TF,JP
0,link link enfp intj moment link sportscenter ...,8,INFJ,2156,0,0,1,0
1,finding lack post alarming sex boring positio...,3,ENTP,3606,1,0,0,1
2,good one link course say know blessing curse ...,11,INTP,2883,0,0,0,1
3,dear intp enjoyed conversation day esoteric g...,10,INTJ,3501,0,0,0,0
4,fired another silly misconception approaching...,2,ENTJ,3253,1,0,0,0


In [112]:
train, test=train_test_split(prep_data,test_size=0.1)
#train=prep_data
print(train.shape)

(7807, 8)


In [114]:
tfidf = TfidfVectorizer(ngram_range=(1,4),analyzer='char',stop_words='english', max_df=0.9, min_df=5)
train_X_tfidf = tfidf.fit_transform(train.content)
print(train_X_tfidf.shape)
print('TFIDF FINISHED...')
test_X_tfidf = tfidf.transform(test.content) #43980
#joblib.dump(tfidf,'lgb/tfidf.pkl')


(7807, 55443)
TFIDF FINISHED...


In [115]:
ha = HashingVectorizer(ngram_range=(1,2),lowercase=False,n_features=2**14)
train_X_ha = ha.fit_transform(train.content)
print(train_X_ha.shape)
test_X_ha = ha.transform(test.content)
#joblib.dump(ha,'lgb/ha.pkl')

mm = MinMaxScaler()

#train_len=mm.fit_transform(train.length.values.reshape(-1,1))
#test_len=mm.transform(test.length.values.reshape(-1,1))
#del mm

train_X=hstack([train_X_tfidf,train_X_ha]) #csr_matrix(train_len)
test_X=hstack([test_X_tfidf,test_X_ha]) #csr_matrix(test_len)

print("hstack finished")

(7807, 16384)
hstack finished


In [116]:
label_col=['IE','NS','TF','JP']
train_Y = train[label_col].values
test_Y = test[label_col].values
print(train_X.shape,test_X.shape)
print(train_Y.shape,test_Y.shape)
train_Y[:5]

(7807, 71827) (868, 71827)
(7807, 4) (868, 4)


array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 1],
       [0, 0, 1, 1]])

In [119]:
'''
I/E classifier
'''
label_index=0
d_train = lgb.Dataset(train_X,train_Y[:,label_index])
d_valid = lgb.Dataset(test_X,test_Y[:,label_index])

watch_list = [d_train,d_valid]

params = {
        'learning_rate': 0.1,
        'objective': 'binary',
        'metric': 'binary_error',
        #'num_class' : 16,
        'num_leaves': 32,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'nthread': 4
    }
#evals_result={}
model1 = lgb.train(params, d_train, 100, watch_list, early_stopping_rounds=15,verbose_eval=1) #25
#online_model1 = lgb.train(params,lgb.Dataset(train_X,train_Y[:,label_index]),num_boost_round=30) # 这里写个300-400效果更好，为了速度问题写的100 # 30


[1]	training's binary_error: 0.231203	valid_1's binary_error: 0.223502
Training until validation scores don't improve for 15 rounds.
[2]	training's binary_error: 0.231203	valid_1's binary_error: 0.223502
[3]	training's binary_error: 0.231203	valid_1's binary_error: 0.223502
[4]	training's binary_error: 0.231203	valid_1's binary_error: 0.223502
[5]	training's binary_error: 0.230178	valid_1's binary_error: 0.223502
[6]	training's binary_error: 0.166005	valid_1's binary_error: 0.177419
[7]	training's binary_error: 0.144614	valid_1's binary_error: 0.15553
[8]	training's binary_error: 0.134367	valid_1's binary_error: 0.147465
[9]	training's binary_error: 0.123607	valid_1's binary_error: 0.144009
[10]	training's binary_error: 0.117843	valid_1's binary_error: 0.142857
[11]	training's binary_error: 0.115665	valid_1's binary_error: 0.140553
[12]	training's binary_error: 0.111054	valid_1's binary_error: 0.139401
[13]	training's binary_error: 0.106699	valid_1's binary_error: 0.138249
[14]	trainin

In [120]:
preds=model1.predict(test_X)
print(preds.shape)
print(f1_score(test_Y[:,label_index],preds>0.5))
print(roc_auc_score(test_Y[:,label_index],preds))

(868,)
0.6784660766961651
0.8781164917862277


In [121]:
'''
N/S Classifier
'''
label_index=1
d_train = lgb.Dataset(train_X,train_Y[:,label_index])
d_valid = lgb.Dataset(test_X,test_Y[:,label_index])

watch_list = [d_train,d_valid]

params = {
        'learning_rate': 0.1,
        'objective': 'binary',
        'metric': 'binary_error',
        #'num_class' : 16,
        'num_leaves': 32,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'nthread': 4
    }
#evals_result={}
model2 = lgb.train(params, d_train, 100, watch_list, early_stopping_rounds=15,verbose_eval=1) #40
#online_model2 = lgb.train(params,lgb.Dataset(train_X,train_Y[:,label_index]),num_boost_round=43) # 这里写个300-400效果更好，为了速度问题写的100


[1]	training's binary_error: 0.138081	valid_1's binary_error: 0.137097
Training until validation scores don't improve for 15 rounds.
[2]	training's binary_error: 0.138081	valid_1's binary_error: 0.137097
[3]	training's binary_error: 0.138081	valid_1's binary_error: 0.137097
[4]	training's binary_error: 0.138081	valid_1's binary_error: 0.137097
[5]	training's binary_error: 0.137569	valid_1's binary_error: 0.137097
[6]	training's binary_error: 0.115025	valid_1's binary_error: 0.117512
[7]	training's binary_error: 0.100551	valid_1's binary_error: 0.114055
[8]	training's binary_error: 0.0913283	valid_1's binary_error: 0.109447
[9]	training's binary_error: 0.0855642	valid_1's binary_error: 0.102535
[10]	training's binary_error: 0.0803125	valid_1's binary_error: 0.0979263
[11]	training's binary_error: 0.0749328	valid_1's binary_error: 0.09447
[12]	training's binary_error: 0.0714743	valid_1's binary_error: 0.0967742
[13]	training's binary_error: 0.0676316	valid_1's binary_error: 0.0956221
[14

In [122]:
preds=model2.predict(test_X)
print(preds.shape)
print(f1_score(test_Y[:,label_index],preds>0.5))
print(roc_auc_score(test_Y[:,label_index],preds))

(868,)
0.5869565217391304
0.9088981386947302


In [123]:
'''
T/F Classifier
'''
label_index=2
d_train = lgb.Dataset(train_X,train_Y[:,label_index])
d_valid = lgb.Dataset(test_X,test_Y[:,label_index])

watch_list = [d_train,d_valid]

params = {
        'learning_rate': 0.1,
        'objective': 'binary',
        'metric': 'binary_error',
        #'num_class' : 16,
        'num_leaves': 32,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'nthread': 4
    }
#evals_result={}
model3 = lgb.train(params, d_train, 100, watch_list, early_stopping_rounds=15,verbose_eval=1) # 65
#online_model3 = lgb.train(params,lgb.Dataset(train_X,train_Y[:,label_index]),num_boost_round=67) # 这里写个300-400效果更好，为了速度问题写的100


[1]	training's binary_error: 0.251953	valid_1's binary_error: 0.267281
Training until validation scores don't improve for 15 rounds.
[2]	training's binary_error: 0.176636	valid_1's binary_error: 0.207373
[3]	training's binary_error: 0.154605	valid_1's binary_error: 0.199309
[4]	training's binary_error: 0.150378	valid_1's binary_error: 0.191244
[5]	training's binary_error: 0.144742	valid_1's binary_error: 0.172811
[6]	training's binary_error: 0.138978	valid_1's binary_error: 0.169355
[7]	training's binary_error: 0.135263	valid_1's binary_error: 0.176267
[8]	training's binary_error: 0.13078	valid_1's binary_error: 0.178571
[9]	training's binary_error: 0.12809	valid_1's binary_error: 0.177419
[10]	training's binary_error: 0.123223	valid_1's binary_error: 0.176267
[11]	training's binary_error: 0.120277	valid_1's binary_error: 0.171659
[12]	training's binary_error: 0.118227	valid_1's binary_error: 0.176267
[13]	training's binary_error: 0.114641	valid_1's binary_error: 0.170507
[14]	training

In [124]:
preds=model3.predict(test_X)
print(preds.shape)
print(f1_score(test_Y[:,label_index],preds>0.5))
print(roc_auc_score(test_Y[:,label_index],preds))

(868,)
0.869022869022869
0.9252843291122912


In [125]:
'''
J/P Classifier
'''
label_index=3
d_train = lgb.Dataset(train_X,train_Y[:,label_index])
d_valid = lgb.Dataset(test_X,test_Y[:,label_index])

watch_list = [d_train,d_valid]

params = {
        'learning_rate': 0.1,
        'objective': 'binary',
        'metric': 'binary_error',
        #'num_class' : 16,
        'num_leaves': 32,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'nthread': 4
    }
#evals_result={}
model4 = lgb.train(params, d_train, 100, watch_list, early_stopping_rounds=15,verbose_eval=1) #30
#online_model4 = lgb.train(params,lgb.Dataset(train_X,train_Y[:,label_index]),num_boost_round=33) # 这里写个300-400效果更好，为了速度问题写的100


[1]	training's binary_error: 0.397464	valid_1's binary_error: 0.381336
Training until validation scores don't improve for 15 rounds.
[2]	training's binary_error: 0.397464	valid_1's binary_error: 0.381336
[3]	training's binary_error: 0.225823	valid_1's binary_error: 0.229263
[4]	training's binary_error: 0.1938	valid_1's binary_error: 0.205069
[5]	training's binary_error: 0.17843	valid_1's binary_error: 0.197005
[6]	training's binary_error: 0.170744	valid_1's binary_error: 0.190092
[7]	training's binary_error: 0.161009	valid_1's binary_error: 0.191244
[8]	training's binary_error: 0.157679	valid_1's binary_error: 0.193548
[9]	training's binary_error: 0.153964	valid_1's binary_error: 0.191244
[10]	training's binary_error: 0.151018	valid_1's binary_error: 0.192396
[11]	training's binary_error: 0.144101	valid_1's binary_error: 0.187788
[12]	training's binary_error: 0.142692	valid_1's binary_error: 0.195853
[13]	training's binary_error: 0.139362	valid_1's binary_error: 0.193548
[14]	training'

In [126]:
preds=model4.predict(test_X)
print(preds.shape)
print(f1_score(test_Y[:,label_index],preds>0.5))
print(roc_auc_score(test_Y[:,label_index],preds))

(868,)
0.8566402814423921
0.8803130291931792


In [35]:
joblib.dump([online_model1,online_model2,online_model3,online_model4],'lgb/lgb_clfs.pkl')

['lgb/lgb_clfs.pkl']

# Label the chat data with MBTI

In [36]:
chat_data=joblib.load('../../data/chat/chat_data.pkl')
print(len(chat_data))
chat_data[0]

2300066


("there's nothing to tell! he's just some guy i work with!",
 "c'mon, you're going out with the guy! there's gotta be something wrong with him!")

In [42]:
def sub_word(posts,remove_stop_words=True):
    temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'link', posts)
    temp = re.sub("[^a-zA-Z]", " ", temp)
    temp = re.sub(' +', ' ', temp).lower()
    if remove_stop_words:
        temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in cachedStopWords])
    else:
        temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
    return temp

In [39]:
def get_proba(word):
    X_tfidf = tfidf.transform(word)
    X_ha = ha.transform(word)
    X=hstack([X_tfidf,X_ha])
    proba=[]
    for clf in [online_model1, online_model2, online_model3, online_model4]:
        proba.append(clf.predict(X)[0])
    return proba

In [81]:
from time import time
t1=time()
label_data=[]
batch_size=10000
batch_start=0
batch_end=batch_start+batch_size
fuck=0
while batch_end < len(chat_data):
    chunck_data=chat_data[batch_start:batch_end]
    ques=[];ans=[];temp=[];
    for dat in chunck_data:
        ques.append(dat[0])
        ans.append(dat[1])
        temp.append(sub_word(dat[1]))
    proba=get_proba(temp)
    new_line=[]
    for ii in range(len(temp)):
        new_line.append((ques[ii],ans[ii],[proba[0][ii],proba[1][ii],proba[2][ii],proba[3][ii]]))
    label_data.extend(new_line)
    batch_start+=batch_size
    batch_end+=batch_size
    fuck+=1
    print(fuck,(time()- t1)/60)

1 0.02560997804005941
2 0.053137771288553876
3 0.07807209094365437
4 0.10307408968607584
5 0.1286808451016744
6 0.1560165246327718
7 0.19099481105804444
8 0.22548333406448365
9 0.26292581160863243
10 0.2971615513165792
11 0.3313321590423584
12 0.36735142866770426
13 0.4033103426297506
14 0.43987718820571897
15 0.4742065866788228
16 0.5090662956237793
17 0.545743179321289
18 0.5774400234222412
19 0.612158203125
20 0.6460737347602844
21 0.6803745269775391
22 0.7136312166849772
23 0.7483024636904398
24 0.781859294573466
25 0.8168975790341695
26 0.8510376652081807
27 0.8860677798589071
28 0.9233596801757813
29 0.9572187662124634
30 0.9910502235094706
31 1.0242370645205179
32 1.0572924375534059
33 1.0914519508679708
34 1.1254964311917623
35 1.1598407824834187
36 1.1955721855163575
37 1.2318145791689554
38 1.2689782818158468
39 1.3054359078407287
40 1.340715487798055
41 1.3790478189786275
42 1.412366251150767
43 1.4477190335591634
44 1.482447592417399
45 1.5169925888379414
46 1.5523545225461

In [82]:
#joblib.dump(label_data,'../../data/chat/labeled_chat_data.pkl')

['../../data/chat/labeled_chat_data.pkl']

In [108]:
sampled_data=[]
for ind in choise_inds:
    sampled_data.append(label_data[ind])
print(len(sampled_data),sampled_data[0])

750000 ('random pic of the day-doctor who-last christmas , 2014 ', "the term last christmas has become a staple point to our christmases . we're more attentive when we see each other each year . ", [0.12544562876859472, 0.3110942006878408, 0.5935377123624724, 0.6965982725652208])


In [109]:
joblib.dump(sampled_data,'../../data/chat/sampled_chat_data.pkl')

['../../data/chat/sampled_chat_data.pkl']

# MBTI Discrimator API with LGB Classifier

In [ ]:
class MBTI():
    def __init__(self, pkl_dir='lgb/'):
        super(MBTI, self).__init__()
        self.models = joblib.load(os.path.join(pkl_dir,'lgb_clfs.pkl'))
        self.ie_clf=self.models[0]
        self.ns_clf=self.models[1]
        self.tf_clf=self.models[2]
        self.jp_clf=self.models[3]
        self.tfidf =  joblib.load(os.path.join(pkl_dir,'tfidf.pkl'))
        self.ha =  joblib.load(os.path.join(pkl_dir,'ha.pkl'))
        
        self.stemmer = PorterStemmer()
        self.lemmatiser = WordNetLemmatizer()
        self.cachedStopWords = stopwords.words("english")
        
        print('Successfully load the model...')
    
    
    def _sub_word(self,posts,remove_stop_words=True):
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'link', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([self.lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in self.cachedStopWords])
        else:
            temp = " ".join([self.lemmatiser.lemmatize(w) for w in temp.split(' ')])
        return temp
    
    def _get_feature(self,raw_x):
        processed=[]
        for item in raw_x:
            processed.append(self._sub_word(item))
            
        X_tfidf = self.tfidf.transform(processed)
        X_ha = self.ha.transform(processed)
        X=hstack([X_tfidf,X_ha])
        return X
    
    @property
    def label_name(self):
        lab_name = ['Introversion (I) – Extroversion (E)',
                    'Intuition (N) – Sensing (S)',
                    'Thinking (T) – Feeling (F)',
                    'Judging (J) – Perceiving (P)']
        return lab_name
        
    def predict(self, x):
        feat=self._get_feature(x)
        ie=self.ie_clf.predict(feat)
        ns=self.ns_clf.predict(feat)
        tf=self.tf_clf.predict(feat)
        jp=self.jp_clf.predict(feat)
        
        proba=[]
        for i,n,t,j in zip(ie,ns,tf,jp):
            proba.append([i,n,t,j])
            
        return np.array(proba)

In [ ]:
mbti=MBTI()
print(mbti.label_name)

In [ ]:
mbti=MBTI()
x=['hello what is your name?',
   'My name is your fathre',
  'haha'
  ]
preds=mbti.predict(x)
print(preds)